In [1]:
import pandas as pd
import numpy.typing as npt
import numpy as np
from math import lgamma, pi, log
from scipy.optimize import minimize
import pymc as pm

# 7.1

In [ ]:
def fit_normal(data: npt.NDArray) -> tuple:
    """
        Given a series of data, return tuple of mean and variance
    """
    n = data.size
    mu = data.sum() / n
    var = ((data - mu) ** 2).sum() / (n - 1)
    return (mu, np.sqrt(var))

## Test

In [ ]:
df = pd.read_csv("test7_1.csv")
df_result = pd.read_csv("testout7_1.csv")
pd.Series(fit_normal(df.x1.to_numpy()))

0    0.046026
1    0.046780
dtype: float64

In [ ]:
df_result.T[0]

mu       0.046026
sigma    0.046780
Name: 0, dtype: float64

# 7.2

In [ ]:
def log_t(x: float, mu: float, sigma: float, nu: float) -> float:
    """"
        The log PDF for T-distribution
    """
    return lgamma((nu + 1) / 2) - lgamma(nu / 2) - 0.5 * log(nu * pi) - log(sigma) - ((nu + 1) / 2) * np.log1p((((x - mu) / sigma) ** 2) / nu)

def neg_loglikelihood(theta, x: npt.NDArray):
    """"
        Give the negative ll for optimizer to minimize
    """
    mu, eta, tau = theta
    sigma = np.exp(eta)
    nu = np.exp(tau) + 2.0
    ll = log_t(x, mu, sigma, nu).sum()
    return -ll
      
      

def fit_T(x: npt.NDArray) -> tuple:
    """
        Given data, return tuple of mu, sigma, nu
    """
    mu0 = np.median(x)
    mad = np.median(np.abs(x - mu0))
    eta0 = np.log(mad)
    tau0 = np.log(10.0 - 2.0)  
    x0 = np.array([mu0, eta0, tau0])
    res = minimize(
        neg_loglikelihood, x0, args=(x,),
        method="L-BFGS-B"
    )
    mu_hat, eta_hat, tau_hat = res.x
    sigma_hat = np.exp(eta_hat)
    nu_hat = np.exp(tau_hat) + 2.0
    return (mu_hat, sigma_hat, nu_hat)


## Test

In [ ]:
df = pd.read_csv("test7_2.csv")
pd.Series(fit_T(df.x1.to_numpy()))

0    0.045940
1    0.045443
2    6.336917
dtype: float64

In [ ]:
pd.read_csv("testout7_2.csv").T[0]

mu       0.045940
sigma    0.045443
nu       6.336875
Name: 0, dtype: float64

# 7.3

In [ ]:
df_in = pd.read_csv("test7_3.csv")

X = np.column_stack([np.ones(len(df_in)), df_in[['x1','x2','x3']].to_numpy()])
y = df_in['y'].to_numpy()
n, p = X.shape 

def student_t_logpdf(e, sigma, nu):
    sigma = np.maximum(sigma, 1e-12)
    nu = np.maximum(nu, 1.999999)  # ensure variance finite
    term1 = lgamma((nu + 1.0)/2.0) - lgamma(nu/2.0)
    term2 = -0.5*(np.log(nu) + np.log(np.pi)) - np.log(sigma)
    term3 = -((nu + 1.0)/2.0)*np.log1p((e/sigma)**2 / nu)
    return term1 + term2 + term3

def nll_t_regression(theta, X, y):
    beta = theta[:p]
    log_sigma = theta[p]
    logit_nu = theta[p+1]  
    lo, hi = 2.0, 100.0
    nu = lo + (hi - lo)/(1.0 + np.exp(-logit_nu))
    sigma = np.exp(log_sigma)
    resid = y - X @ beta
    return -np.sum(student_t_logpdf(resid, sigma, nu))

beta_ols, *_ = np.linalg.lstsq(X, y, rcond=None)
resid_ols = y - X @ beta_ols
sigma0 = np.std(resid_ols)
nu0 = 5.0
lo, hi = 2.0, 100.0
z0 = (nu0 - lo)/(hi - lo)
logit_nu0 = np.log(z0) - np.log(1 - z0)

theta0 = np.r_[beta_ols, np.log(sigma0), logit_nu0]

res = minimize(nll_t_regression, theta0, args=(X, y), method="L-BFGS-B")
theta_hat = res.x

beta_hat = theta_hat[:p]
log_sigma_hat = theta_hat[p]
logit_nu_hat = theta_hat[p+1]
sigma_hat = float(np.exp(log_sigma_hat))
nu_hat = 2.0 + (100.0 - 2.0)/(1.0 + np.exp(-logit_nu_hat))

beta_names = ['Alpha','B1','B2','B3']
est_row = {
    'mu_assumed': 0.0,
    'sigma_hat': sigma_hat,
    'nu_hat': nu_hat,
}
for name, val in zip(beta_names, beta_hat):
    est_row[name] = float(val)

pd.Series(est_row)

mu_assumed    0.000000
sigma_hat     0.048548
nu_hat        4.598466
Alpha         0.042634
B1            0.974810
B2            2.041188
B3            3.154821
dtype: float64

In [ ]:
pd.read_csv("testout7_3.csv").T[0]

mu       0.000000
sigma    0.048548
nu       4.598293
Alpha    0.042634
B1       0.974889
B2       2.041192
B3       3.154801
Name: 0, dtype: float64

In [2]:
1/0.04

25.0